<a href="https://colab.research.google.com/github/VuDLance/colab/blob/main/aai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import random
import cv2
import os
import tensorflow as tf
from tqdm import tqdm

In [ ]:
good_frames = 'C:/Users/OS/Pictures/ReCover'
bad_frames = 'C:/Users/OS/Pictures/Saved Pictures'

In [ ]:
clean_frames = []
for file in tqdm (sorted (os.listdir (good_frames))):
  if any(extension in file for extension in ['.jpg', '.jpeg', '.png']):
         image = tf.keras.preprocessing.image.load_img(good_frames + '/' + file, target_size=(128,128))
         image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
          clean_frames.append(image)

clean_frames = np.array(clean_frames)
blurry_frames = []
for file in tqdm (sorted (os.listdir(bad_frames))):
    if any (extension in file for extension in ['.jpg', 'jpeg', '.png']):
        image = tf.keras.preprocessing.image.load_img (bad_frames + '/' + file, target_size=(128,128))
        image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
        blurry_frames.append(image)

blurry_frames = np.array(blurry_frames)


IndentationError: ignored

In [ ]:
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras import backend as K

seed = 21
random.seed
seed
np.random.seed = seed


In [ ]:
x = clean_frames;
y = blurry_frames;
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
print(x_train[0].shape)
print(y_train[0].shape)

In [ ]:
r = random.randint(0, len(clean_frames)-1)
print(r)
fig = plt.figure()
fig.subplots_adjust (hspace=0.1, wspace=0.2)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(clean_frames[r])
ax = fig.add_subplot(1, 2, 2)
ax.imshow(blurry_frames[r])

In [ ]:
#Network Parameters
input_shape = (128, 128, 3)
batch_size = 32
kernel_size = 3
latent_dim= 256
# Encoder/Decoder number of CNN layers and filters per layer
layer_filters = [64, 128, 256]


In [ ]:
inputs = Input(shape = input_shape, name= 'encoder_input')
x = inputs

In [ ]:
for filters in layer_filters:
  x = Conv2D(filters=filters,
              kernel_size=kernel_size,
              strides=2,
              activation='relu',
              padding='same')(x)
shape = K.int_shape(x)
x = Flatten()(x)
latent = Dense(latent_dim, name='latent_vector')(x)


In [ ]:
encoder = Model(inputs, latent, name='encoder')
encoder.summary()

In [ ]:
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(shape[1]*shape [2]*shape [3])(latent_inputs)
x = Reshape((shape [1], shape [2], shape [3]))(x)
for filters in layer_filters [::-1):
    x = Conv2DTranspose(filters-filters,
                        kernel_size=kernel_size,
                        strides=2,
                        activation='relu',
                        padding='same')(x)
outputs = Conv2DTranspose(filters=3,
                          kernel_size=kernel_size,
                          activation='sigmoid',
                          padding='same',
                          name='decoder_output')(x)

In [ ]:
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

In [ ]:
autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')
autoencoder.summary()

In [ ]:
autoencoder.compile(loss='mse', optimizer='adam', metrics=["acc"])

In [ ]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                cooldown=0,
                                patience=5,
                                verbose=1,
                                min_lr=0.5e-6)


In [ ]:
callbacks = [lr_reducer]
history = autoencoder.fit(blurry_frames,
                          clean_frames,
                          validation_data=(blurry_frames, clean_frames),
                          epochs=100,
                          batch_size=batch_size,
                          callbacks=callbacks)

In [ ]:
print("\n 	Input		Ground Truth			Predicted Value")

for i in range(3):

r = random.randint(0, len(clean_frames)-1)

x, y = blurry_frames [r], clean_frames [r]
x_inp=x.reshape(1,128,128,3)
result = autoencoder.predict(x_inp)
result = result.reshape(128,128,3)

fig = plt.figure(figsize=(12,10))
fig.subplots_adjust(hspace=0.1, wspace=0.2)

ax = fig.add_subplot(1, 3, 1)
ax.imshow(x)

ax = fig.add_subplot(1, 3, 2)
ax.imshow(y)

ax = fig.add_subplot(1, 3, 3)
plt.imshow(result)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(np.arange(0, 101, 25))
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(np.arange(0, 101, 25))
plt.show()